importing all relevant libraries

In [45]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols

reading in csv file containing all the relevent features we are interested in to explore and find a model

In [56]:
df = pd.read_csv('our_dataframe.csv', index_col=0)

In [57]:
df.columns

Index(['saleprice', 'sqfttotliving', 'yrbuilt', 'yrrenovated', 'bedrooms',
       'zipcode', 'sqfttotbasement', 'sqftfinbasement', 'sqftopenporch',
       'sqftenclosedporch', 'sqftdeck', 'heatsystem', 'heatsource',
       'bathhalfcount', 'bath3qtrcount', 'bathfullcount', 'condition',
       'viewutilization', 'sqftgarageattached', 'daylightbasement',
       'finbasementgrade', 'hbuasifvacant', 'inadequateparking', 'mtrainier',
       'olympics', 'cascades', 'territorial', 'seattleskyline', 'pugetsound',
       'lakewashington', 'lakesammamish', 'smalllakerivercreek', 'otherview',
       'wfntlocation', 'trafficnoise', 'airportnoise', 'powerlines',
       'othernuisances', 'adjacentgreenbelt'],
      dtype='object')

Adding in bath, porch, price per square foot in data frame and dropping individual columns containing parts of these newly added columns

In [58]:
df['porch'] = df['sqftopenporch'] + df['sqftenclosedporch']
df['baths'] = df['bathfullcount'] + (0.75*df['bath3qtrcount']) + (0.5*df['bathhalfcount'])
df.drop(columns=['sqftopenporch', 'sqftenclosedporch', 'bathfullcount', 'bath3qtrcount',
                 'bathhalfcount'], inplace=True)

In [59]:
df['pricepersqft'] = df['saleprice']/df['sqfttotliving']
df['pricepersqft_finbasement'] = df['saleprice']/ (df['sqfttotliving'] + df['sqftfinbasement'])

In [60]:
df.shape


(18293, 38)

18293 entries at present, removing outliers in saleprice that are 3 sigma away from the mean

In [51]:
mean, std = df['saleprice'].agg(['mean', 'std'])
def z_score(row):
    return (row - mean) / std

df['z_score'] = df.saleprice.apply(z_score)
df = df[abs(df['z_score']) < 3]
df

,saleprice,sqfttotliving,yrbuilt,yrrenovated,bedrooms,zipcode,sqfttotbasement,sqftfinbasement,sqftdeck,heatsystem,...,trafficnoise,airportnoise,powerlines,othernuisances,adjacentgreenbelt,porch,baths,pricepersqft,pricepersqft_finbasement,z_score
0,1910000,3990,1952,2019,6,98125,1360,1360,1070,7,...,0,0,N,N,N,110,4.50,478.696742,357.009346,1.902353
1,1089950,3920,2018,0,5,98028.0,930,930,0,5,...,0,0,N,N,N,550,3.25,278.048469,224.731959,0.503503
2,412500,2280,1958,0,4,98056,1140,1140,0,7,...,1,0,N,N,N,0,1.75,180.921053,120.614035,-0.652098
3,541000,1030,1965,0,3,98052,0,0,0,5,...,0,0,N,N,N,210,1.50,525.242718,525.242718,-0.432901
4,1020000,2410,1939,0,3,98144,1070,1070,170,5,...,1,0,N,N,N,0,1.75,423.236515,293.103448,0.384182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18288,1201000,3680,1974,0,5,98072,0,0,1000,5,...,0,0,N,N,N,50,3.00,326.358696,326.358696,0.692933
18289,921000,2280,2016,0,3,NaN,0,0,0,5,...,0,0,N,N,N,110,2.50,403.947368,403.947368,0.215306
18290,627000,3820,1967,0,5,98058,0,0,0,5,...,0,0,N,N,N,80,2.25,164.136126,164.136126,-0.286202
18291,640000,2450,2015,0,4,98146,0,0,0,5,...,2,0,N,N,N,50,2.50,261.224490,261.224490,-0.264026


In [52]:
outliers_removed = 18293 - 17990
print('the outliers that were 3 sigma away from the mean related to ', outliers_removed, ' entries')

the outliers that were 3 sigma away from the mean related to  303  entries


In [54]:
df['saleprice'].describe()

count    1.799000e+04
mean     7.452165e+05
std      3.909353e+05
min      1.000000e+01
25%      4.690000e+05
50%      6.500000e+05
75%      8.899970e+05
max      2.550000e+06
Name: saleprice, dtype: float64

In [55]:
mean, std = df['saleprice'].agg(['mean', 'std'])
print(mean, std)

745216.4739855475 390935.27068573574


In [33]:
df.columns

Index(['saleprice', 'sqfttotliving', 'yrbuilt', 'yrrenovated', 'bedrooms',
       'zipcode', 'sqfttotbasement', 'sqftfinbasement', 'sqftdeck',
       'heatsystem', 'heatsource', 'condition', 'viewutilization',
       'sqftgarageattached', 'daylightbasement', 'finbasementgrade',
       'hbuasifvacant', 'inadequateparking', 'mtrainier', 'olympics',
       'cascades', 'territorial', 'seattleskyline', 'pugetsound',
       'lakewashington', 'lakesammamish', 'smalllakerivercreek', 'otherview',
       'wfntlocation', 'trafficnoise', 'airportnoise', 'powerlines',
       'othernuisances', 'adjacentgreenbelt', 'porch', 'baths', 'pricepersqft',
       'pricepersqft_finbasement'],
      dtype='object')

In [40]:
df_corr = df.corr()['saleprice'].sort_values(ascending=False)

In [42]:
len(df_corr)

32

In [49]:
df_corr[abs(df_corr) > 0.2]

saleprice                   1.000000
sqfttotliving               0.621515
baths                       0.495995
pricepersqft                0.480202
pricepersqft_finbasement    0.415050
lakewashington              0.378633
territorial                 0.312686
sqftfinbasement             0.291167
sqfttotbasement             0.272623
bedrooms                    0.271100
porch                       0.270791
wfntlocation                0.261991
finbasementgrade            0.234935
sqftdeck                    0.208910
Name: saleprice, dtype: float64

choosing the highest correlation with saleprice and also choosing only the features of the property for now (will look into the location aspects which is the key in real estate as we progress)

sqfttotliving 0.621515 vs pricepersqft 0.480202 vs pricepersqft_finbasement 0.415050 (essentially the same thing in different forms, can include the one that best suits the R-squareed and works with other predictors)

baths 0.495995

sqftfinbasement 0.291167, sqfttotbasement 0.272623

bedrooms  0.271100

porch 0.270791

lakewashington view 0.378633, territorial view 0.312686 (the codes used in these two range from 1-4 relating to fair - excellent view)

wfntlocation 0.261991 (this should be more of a yes and no type field since the codes used for this variable only categorizes a unit having 1 of 9 different waterfront areas in king county, need to look into this more carefully)

finished basement grade (1-20 relating to )

checking for linearity between the chosen predictors

In [48]:
df['viewutilization'].value_counts()

     9625
N    8721
Y     425
Name: viewutilization, dtype: int64

In [1]:
df['saleprice']

NameError: name 'df' is not defined